In [1]:
from datasets import load_dataset
import glob
import os
import torch
import torchaudio

In [2]:
sources = glob.glob("MYRiAD_V2/audio/SRC/*.wav")
cocktail_party = [f"CP{i}.wav" for i in range(6)];
source_list = [suffix[-6:] for suffix in sources] + cocktail_party 
inner = [f"CMA10_{a}_" for a in [0,90,180,-90]]
outer = [f"CMA20_{a}_" for a in [0,45,90,135,180,-135,-90,-45]]
channel_list = inner + outer
ref_channel = channel_list[0]

In [3]:
if not os.path.exists("CMA12"):
    os.makedirs("CMA12")

for source in source_list:
    query = ref_channel+source
    prefix_list = glob.glob("MYRiAD_V2/audio/*/*/*/"+query)
    prefix_list = [match[:-len(query)] for match in prefix_list]
    for prefix in prefix_list:
        x = [];
        try:
            for channel in channel_list:
                filename = prefix + channel + source
                x.append(torchaudio.load(filename)[0][0])
            x = torch.stack(x)
            new_file = "CMA12/"+filename[16:].replace("/", "_")
            torchaudio.save(new_file,x,44100)
        except:
            pass

In [5]:
dataset = load_dataset("audiofolder", data_dir="CMA12")

Resolving data files:   0%|          | 0/149 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset.push_to_hub("danjacobellis/CMA12",max_shard_size="100MB")

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/168 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]